In [26]:
import numpy as np
import os
from imageio import imread
from PIL import Image
import datetime
import os

In [27]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [28]:
cwd = os.getcwd()
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 32

In [29]:
import numpy as np
import os
from imageio import imread  # Make sure scipy is installed as mentioned
from PIL import Image
import cv2  # For image processing

def generator(source_path, folder_list, batch_size):
    print('Source path =', source_path, '; batch size =', batch_size)
    img_idx = list(range(0, 30))
    # Get number of batches
    num_batches = len(folder_list) // batch_size
    x = 30
    y = 180
    z = 180
    while True:
        t = np.random.permutation(folder_list)  # Shuffle the folder list
        
        for batch in range(num_batches):  # Iterate over the number of batches
            batch_data = np.zeros((batch_size, x, y, z, 3))  # Create a placeholder for batch data (x, y, z, 3 channels)
            batch_labels = np.zeros((batch_size, 5))  # One-hot encoded labels (for 5 classes)

            for folder in range(batch_size):  # Iterate over the batch_size
                folder_path = os.path.join(source_path, t[folder + (batch * batch_size)].split(';')[0])
                imgs = os.listdir(folder_path)  # Read all images in the folder
                
                # Iterate over the frames/images in the folder (based on `img_idx`)
                for idx, item in enumerate(img_idx):
                    image = imread(os.path.join(folder_path, imgs[item])).astype(np.float32)

                    # Crop and resize images to ensure consistent shape (y, z)
                    image = cv2.resize(image, (z, y))  # Resize to (y, z) shape, ensure the right shape for Conv3D

                    # Normalize and feed in the image
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] / 255.0  # Normalize RGB channels
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] / 255.0
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] / 255.0

                # Assign label (one-hot encoding)
                label_index = int(t[folder + (batch * batch_size)].strip().split(';')[2])
                batch_labels[folder, label_index] = 1
            
            yield batch_data, batch_labels  # Yield the batch data and labels

        # Handle the remaining data points after full batches
        remaining_samples = len(folder_list) % batch_size
        if remaining_samples > 0:
            batch_data = np.zeros((remaining_samples, x, y, z, 3))
            batch_labels = np.zeros((remaining_samples, 5))

            for folder in range(remaining_samples):
                folder_path = os.path.join(source_path, t[folder + (num_batches * batch_size)].split(';')[0])
                imgs = os.listdir(folder_path)

                for idx, item in enumerate(img_idx):
                    image = imread(os.path.join(folder_path, imgs[item])).astype(np.float32)

                    # Crop and resize images to ensure consistent shape (y, z)
                    image = cv2.resize(image, (z, y))

                    # Normalize and feed in the image
                    batch_data[folder, idx, :, :, 0] = image[:, :, 0] / 255.0
                    batch_data[folder, idx, :, :, 1] = image[:, :, 1] / 255.0
                    batch_data[folder, idx, :, :, 2] = image[:, :, 2] / 255.0

                # Assign label (one-hot encoding)
                label_index = int(t[folder + (num_batches * batch_size)].strip().split(';')[2])
                batch_labels[folder, label_index] = 1

            yield batch_data, batch_labels  # Yield the remaining batch data and labels


In [30]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# Hyperparameters
img_size = (180, 180)  # Resize images
frames = 30  # Number of frames per sequence
learning_rate = 0.001

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [31]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, GRU, Dense, Dropout, TimeDistributed, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import SpatialDropout2D


def cnn_rnn_model(input_shape, num_classes):
    model = Sequential()

    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape[1:])
    base_model.trainable = False  # Freeze ResNet50 to prevent overfitting initially
    

    model.add(TimeDistributed(base_model, input_shape=input_shape))
    model.add(TimeDistributed(SpatialDropout2D(0.3)))
    model.add(TimeDistributed(Flatten()))  

    model.add(GRU(32, return_sequences=False, activation='relu'))
    model.add(Dropout(0.5)) 

   
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())  
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))  

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

model = cnn_rnn_model(input_shape=(30, 180, 180, 3), num_classes=5)





In [32]:
# from tensorflow.keras.applications import MobileNetV2
# from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
# from tensorflow.keras.models import Sequential
# def create_model(input_shape, num_classes):
#     base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape[1:])
#     base_model.trainable = False  # Freeze layers

#     model = Sequential([
#         TimeDistributed(base_model, input_shape=input_shape),
#         TimeDistributed(Flatten()),
#         GRU(16, return_sequences=False),
#         Dense(num_classes, activation='softmax')
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
#     return model
# model = create_model(input_shape=(30, 180, 180, 3), num_classes=5)

In [33]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDis  (None, 30, 6, 6, 1280)   2257984   
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 30, 6, 6, 1280)   0         
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 30, 46080)        0         
 tributed)                                                       
                                                                 
 gru_2 (GRU)                 (None, 32)                4426944   
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                      

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [35]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{accuracy:.5f}-{val_loss:.5f}-{val_accuracy:.5f}.keras'

filepath = model_name + "model{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras"


checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')


LR = ReduceLROnPlateau(monitor='val_loss',  
                       factor=0.5,           
                       patience=5,           
                       verbose=1,            
                       min_lr=1e-6)          

callbacks_list = [checkpoint, LR, early_stop]

In [36]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [37]:
model.fit(
    train_generator,                
    steps_per_epoch=steps_per_epoch, 
    epochs=num_epochs,              
    verbose=1,                        
    callbacks=callbacks_list,        
    validation_data=val_generator,   
    validation_steps=validation_steps                
)

Source path = /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - ETA: 0s - loss: 2.2796 - categorical_accuracy: 0.2157Source path = /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_init_2025-03-0316_18_51.841788/model00001-2.27965-0.21569-2.79084-0.16000.keras
21/21 [==============================] - 87s 4s/step - loss: 2.2796 - categorical_accuracy: 0.2157 - val_loss: 2.7908 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/30
21/21 [==============================] - ETA: 0s - loss: 2.2054 - categorical_accuracy: 0.2112
Epoch 00002: saving model to model_init_2025-03-0316_18_51.841788/model00002-2.20545-0.21116-1.60139-0.30000.keras
21/21 [==============================] - 77s 4s/step - loss: 2.2054 - categorical_accuracy: 0.2112 - val_loss: 1.6014 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 3/30
21/21 [==============================] - ETA: 0s - loss: 1.9891 - categorical_accurac

In [38]:
scores = model.evaluate(val_generator, 
verbose=1, 
steps=validation_steps*10, 
max_queue_size=3000, 
workers=1, 
use_multiprocessing=False)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

40/40 [==============================] - 113s 3s/step - loss: 0.5560 - categorical_accuracy: 0.8130
evaluate categorical_accuracy: 81.30%
